In [1]:
import os, io
import numpy as np
import pandas as pd
from PIL import Image
import bson
import time

%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
def make_category_tables():
    cat2idx = {}
    idx2cat = {}
    for ir in categories_df.itertuples():
        category_id = ir[0]
        category_idx = ir[4]
        cat2idx[category_id] = category_idx
        idx2cat[category_idx] = category_id
    return cat2idx, idx2cat

categories_df = pd.read_csv("categories.csv", index_col=0)
cat2idx, idx2cat = make_category_tables()

train_offsets_df = pd.read_csv("train_offsets.csv", index_col=0)
train_images_df = pd.read_csv("train_images.csv", index_col=0)
val_images_df = pd.read_csv("val_images.csv", index_col=0)

test_offsets_df = pd.read_csv("test_offsets.csv", index_col=0)
test_images_df = pd.read_csv("test_images.csv", index_col=0)

num_train_images = len(train_images_df)
num_val_images = len(val_images_df)
num_test_images = len(test_images_df)
print('num_train_images = %d, num_val_images = %d, num_test_images = %d' % (num_train_images, num_val_images, num_test_images))

data_dir = "/mnt/data/cdiscount/"

train_bson_path = os.path.join(data_dir, "train.bson")
test_bson_path = os.path.join(data_dir, "test.bson")

train_bson_file = open(train_bson_path, "rb")
test_bson_file = open(test_bson_path, "rb")

batch_size = 100

/home/weiso/tensorflow/lib/python3.5/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


num_train_images = 9900946, num_val_images = 2470347, num_test_images = 3095080


In [5]:
image_shape = (128, 128)
num_class = 5270

def get_batch(bson_file, images_df, offsets_df, index_array, with_labels):
    batch_x = np.zeros((len(index_array),) + image_shape + (3, ), dtype=np.float16)
    batch_id = np.zeros(len(index_array), dtype=np.uint32)
    if with_labels:
        batch_y = np.zeros((len(batch_x), num_class), dtype=np.float16)

    for i, j in enumerate(index_array):
        image_row = images_df.iloc[j]
        product_id = image_row["product_id"]
        offset_row = offsets_df.loc[product_id]

        # Read this product's data from the BSON file.
        bson_file.seek(offset_row["offset"])
        item_data = bson_file.read(offset_row["length"])

        # Grab the image from the product.
        item = bson.BSON.decode(item_data)
        img_idx = image_row["img_idx"]
        bson_img = item["imgs"][img_idx]["picture"]

        # Preprocess the image.
        img = Image.open(io.BytesIO(bson_img))
        img = img.resize(image_shape)
        x = np.asarray(img, dtype=np.float16)
        
        #x = self.image_data_generator.random_transform(x)
        #x = self.image_data_generator.standardize(x)

        # Add the image and the label to the batch (one-hot encoded).
        batch_x[i] = x
        batch_id[i] = product_id
        if with_labels:
            batch_y[i, image_row["category_idx"]] = 1

    if with_labels:
        return batch_x, batch_y, batch_id
    else:
        return batch_x, batch_id



In [9]:
train_index_array_all = np.arange(num_train_images)
np.random.shuffle(train_index_array_all)

val_index_array_all = np.arange(num_val_images)
np.random.shuffle(val_index_array_all)

train_batch_index_array

train_batch_x, train_batch_y, _ = get_batch(train_bson_file, train_images_df, train_offsets_df, train_batch_index_array, with_labels=True)
